In [45]:
from sklearn.datasets import load_iris, load_diabetes,load_wine
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score,mean_squared_error
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold,cross_val_score, GridSearchCV, RandomizedSearchCV,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from statistics import mean
from hpsklearn import HyperoptEstimator, svc,any_classifier
from hyperopt import tpe

In [29]:
wine = load_wine()

features, target = wine.data, wine.target

features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

logistic_reg = LogisticRegression(random_state=0,max_iter=5000,solver="liblinear",penalty="l1",C=2.0).fit(features_train,target_train)
logistic_reg.score(features_test, target_test)


0.9111111111111111

In [30]:
C_value= np.linspace(1.0,10000.0,num=1000,dtype=float)

In [31]:

parameters = {'penalty':('l1', 'l2'), 'C':C_value}
grid_search = GridSearchCV(logistic_reg, parameters,verbose=1,n_jobs=-2)
grid_search.fit(features_train,target_train)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


GridSearchCV(estimator=LogisticRegression(C=2.0, max_iter=5000, penalty='l1',
                                          random_state=0, solver='liblinear'),
             n_jobs=-2,
             param_grid={'C': array([1.00000000e+00, 1.10090090e+01, 2.10180180e+01, 3.10270270e+01,
       4.10360360e+01, 5.10450450e+01, 6.10540541e+01, 7.10630631e+01,
       8.10720721e+01, 9.10810811e+01, 1.01090090e+02, 1.11099099e+02,
       1.21108108e+02, 1.31117117e+...
       9.76979279e+03, 9.77980180e+03, 9.78981081e+03, 9.79981982e+03,
       9.80982883e+03, 9.81983784e+03, 9.82984685e+03, 9.83985586e+03,
       9.84986486e+03, 9.85987387e+03, 9.86988288e+03, 9.87989189e+03,
       9.88990090e+03, 9.89990991e+03, 9.90991892e+03, 9.91992793e+03,
       9.92993694e+03, 9.93994595e+03, 9.94995495e+03, 9.95996396e+03,
       9.96997297e+03, 9.97998198e+03, 9.98999099e+03, 1.00000000e+04]),
                         'penalty': ('l1', 'l2')},
             verbose=1)

In [32]:
#the best parameters for grid search
print(grid_search.best_estimator_.get_params()['penalty'])
print(grid_search.best_estimator_.get_params()['C'])


l2
11.00900900900901


In [33]:
import os
print(os.cpu_count())

8


In [34]:
parameters = {'penalty':('l1', 'l2'), 'C':C_value}
Randomized_search = RandomizedSearchCV(logistic_reg, parameters,verbose=1,n_jobs=-1,n_iter=100)
Randomized_search.fit(features_train,target_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(estimator=LogisticRegression(C=2.0, max_iter=5000,
                                                penalty='l1', random_state=0,
                                                solver='liblinear'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e+00, 1.10090090e+01, 2.10180180e+01, 3.10270270e+01,
       4.10360360e+01, 5.10450450e+01, 6.10540541e+01, 7.10630631e+01,
       8.10720721e+01, 9.10810811e+01, 1.01090090e+02, 1.11099099e+02,
       1...
       9.76979279e+03, 9.77980180e+03, 9.78981081e+03, 9.79981982e+03,
       9.80982883e+03, 9.81983784e+03, 9.82984685e+03, 9.83985586e+03,
       9.84986486e+03, 9.85987387e+03, 9.86988288e+03, 9.87989189e+03,
       9.88990090e+03, 9.89990991e+03, 9.90991892e+03, 9.91992793e+03,
       9.92993694e+03, 9.93994595e+03, 9.94995495e+03, 9.95996396e+03,
       9.96997297e+03, 9.97998198e+03, 9.98999099e+03, 1.00000000e+04]),
                                        'pe

In [35]:
#the best parameters for grid search
print(Randomized_search.best_estimator_.get_params()['penalty'])
print(Randomized_search.best_estimator_.get_params()['C'])

l2
831.7477477477478


In [36]:
logistic_reg_2 = LogisticRegression(random_state=0,max_iter=5000,solver="liblinear",penalty="l1",C=2.0)

pipe = Pipeline([("classifier",RandomForestClassifier())])

search_space = [
                {
                "classifier":[logistic_reg_2],
                "classifier__penalty":['l1','l2'],
                "classifier__C":C_value
                },
                {
                "classifier":[RandomForestClassifier()],
                "classifier__n_estimators":[10,100,1000],
                "classifier__max_features":[1,2,3]
                },
                {
                "classifier":[KNeighborsClassifier()],
                "classifier__n_neighbors":[3,5,7]
                }
                ]

In [37]:
multi_grid_search = GridSearchCV(pipe, search_space,verbose=1,n_jobs=-2)
best_modele = multi_grid_search.fit(features_train,target_train)

Fitting 5 folds for each of 2012 candidates, totalling 10060 fits


In [52]:
estim = HyperoptEstimator(classifier=any_classifier("my_clf"),
                              preprocessing=[],
                              max_evals=17,
                              n_jobs=-1,
                              algo=tpe.suggest,
                              trial_timeout=300)

    # Search the hyperparameter space based on the data
estim.fit(features_train, target_train)

    # Show the results
print(estim.score(features_test, target_test))

100%|██████████| 17/17 [00:02<00:00,  2.86s/trial, best loss: 0.03703703703703709]
0.9777777777777777
